In [1]:
import numpy as np
import pandas as pd
import datetime
import copy
import time
import os
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import operator

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
from datetime import timedelta

tqdm.pandas()

In [2]:
# Edit to point to your MIMIC directory.
dataDirStr = '/Users/gmessier/data/mimic-1.4/'

In [3]:
services_df = pd.read_csv(dataDirStr + "SERVICES.csv")
services_df.columns = services_df.columns.str.lower()
services_df

,row_id,subject_id,hadm_id,transfertime,prev_service,curr_service
0,758,471,135879,2122-07-22 14:07:27,TSURG,MED
1,759,471,135879,2122-07-26 18:31:49,MED,TSURG
2,760,472,173064,2172-09-28 19:22:15,NaN,CMED
3,761,473,129194,2201-01-09 20:16:45,NaN,NB
4,762,474,194246,2181-03-23 08:24:41,NaN,NB
...,...,...,...,...,...,...
73338,72914,98932,174244,2176-08-13 20:28:00,NaN,CMED
73339,72915,98939,115549,2166-11-15 01:21:49,NaN,NMED
73340,72916,98941,141129,2118-02-08 01:52:28,NaN,CSURG
73341,72917,98943,193747,2164-11-14 20:04:12,NaN,TRAUM


The `SERVICES` table describes the service that a patient was admitted under. While a patient can be physicially located at a given ICU type (say MICU), they are not necessarily being cared for by the team which staffs the MICU. This can happen due to a number of reasons, including bed shortage. The `SERVICES` table should be used if interested in identifying the type of service a patient is receiving in the hospital.

`transfertime` is the time at which the patient moved from the `prev_service` (if present) to the `curr_service`

Each service in the table is listed as an abbreviation. The descriptions for each of the possible services are as follows:

`CMED`	Cardiac Medical - for non-surgical cardiac related admissions

`CSURG`	Cardiac Surgery - for surgical cardiac admissions

`DENT`	Dental - for dental/jaw related admissions

`ENT`	Ear, nose, and throat - conditions primarily affecting these areas

`GU`	Genitourinary - reproductive organs/urinary system

`GYN`	Gynecological - female reproductive systems and breasts

`MED`	Medical - general service for internal medicine

`NB`	Newborn - infants born at the hospital

`NBB`	Newborn baby - infants born at the hospital

`NMED`	Neurologic Medical - non-surgical, relating to the brain

`NSURG`	Neurologic Surgical - surgical, relating to the brain

`OBS`	Obstetrics - conerned with childbirth and the care of women giving birth

`ORTHO`	Orthopaedic - surgical, relating to the musculoskeletal system

`OMED`	Orthopaedic medicine - non-surgical, relating to musculoskeletal system

`PSURG`	Plastic - restortation/reconstruction of the human body (including cosmetic 
or aesthetic)

`PSYCH`	Psychiatric - mental disorders relating to mood, behaviour, cognition, or perceptions

`SURG`	Surgical - general surgical service not classified elsewhere

`TRAUM`	Trauma - injury or damage caused by physical harm from an external source

`TSURG`	Thoracic Surgical - surgery on the thorax, located between the neck and the abdomen

`VSURG`	Vascular Surgical - surgery relating to the circulatory system

`prev_service` and `curr_service` are the previous and current service that the patient resides under.

In [4]:
c = services_df.prev_service.value_counts()[:5]
p = services_df.prev_service.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
MED,5001,34.09
CMED,3655,24.92
OMED,1066,7.27
SURG,993,6.77
NSURG,655,4.47


In [5]:
c = services_df.curr_service.value_counts()[:5]
p = services_df.curr_service.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
MED,24866,33.90
CMED,9135,12.46
CSURG,7911,10.79
NB,7806,10.64
SURG,5514,7.52
